In [675]:
import time
start_time = time.time()
import numpy as np
import pandas as pd
import nltk, re, pprint
from __future__ import division

In [718]:
# read the training data
df_train = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/train.csv", encoding="ISO-8859-1") #update here
df_test = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/test.csv", encoding="ISO-8859-1") #update here
df_att = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/attributes.csv", encoding="ISO-8859-1") #update here
df_pro = pd.read_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/Data/product_descriptions.csv", encoding="ISO-8859-1") #update here

In [719]:
df_train_df = pd.DataFrame(df_train)
df_train_pro = pd.DataFrame(df_pro)
df_test_df = pd.DataFrame(df_test)

In [720]:
result = pd.merge(df_train_df, df_train_pro, on='product_uid')
result_test = pd.merge(df_test_df, df_train_pro, on='product_uid')

In [721]:
# define the function for splitting the words
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

In [726]:
# add a new column called search_split
ss = result["search_term"].apply(split_search) 
result['search_split'] = pd.Series(ss, index=result.index)
ss = result_test["search_term"].apply(split_search) 
result_test['search_split'] = pd.Series(ss, index=result_test.index)

In [728]:
# help to split the string
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1
nrow = result.shape[0]
new_search_split = [0]*nrow
for index, row in result.iterrows():
    each_word_len = [len(w) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result['new_search_split'] = pd.Series(new_search_split, index=result.index)

# help to split the string for test data
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

# calculate the word length for each of the searching words
# remove the word that has length 1

nrow = result_test.shape[0]
new_search_split = [0]*nrow
for index, row in result_test.iterrows():
    each_word_len = [len(str(w)) for w in row["search_split"]]
    row["search_split"] = [word for word in row["search_split"] if len(word) > 1]
    new_search_split[index] = row["search_split"]
    each_word_len = [mm for mm in each_word_len if mm > 1]

# add a new column called search_split
result_test['new_search_split'] = pd.Series(new_search_split, index=result_test.index)

In [730]:
# find how many words in the product description match the searching query 
nrow = result.shape[0]
desc_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    desc_match_word[index] = sum(ll)

# add a new column called description match words
result['desc_match_word'] = pd.Series(desc_match_word, index=result.index)


In [733]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [734]:
#result_test["product_description"][73024] = result_test["product_description"][1]
#result_test["new_search_split"][73024] = result_test["new_search_split"][1]
#118900
# 118901
# find how many words in the product description match the searching query 
nrow = result_test.shape[0]
desc_match_word = [0]*nrow
for index, row in result_test.iterrows():    
    ll = [len(re.findall(w, row["product_description"])) for w in row["new_search_split"]]
    print (ll)
    desc_match_word[index] = sum(ll)
result_test['desc_match_word'] = pd.Series(desc_match_word, index=result_test.index)


In [735]:
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

In [736]:
# find how many words match in the product title
nrow = result.shape[0]
title_match_word = [0]*nrow
for index, row in result.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result['title_match_word'] = pd.Series(title_match_word, index=result.index)

# test data
# find how many words match in the product title
nrow = result_test.shape[0]
title_match_word = [0]*nrow
for index, row in result_test.iterrows():
    ll = [len(re.findall(w, str(row["product_title"]).lower())) for w in row["new_search_split"]]
    print (ll)
    print(index)
    title_match_word[index] = sum(ll)

# add a new column called description match words
result_test['title_match_word'] = pd.Series(title_match_word, index=result_test.index)


In [737]:
# build random forest model

from sklearn.ensemble import RandomForestClassifier
cols = ['desc_match_word', 'title_match_word'] 
colsRes = ['relevance']
trainArr = result.as_matrix(cols) #training array
trainRes = result.as_matrix(colsRes) # training results
rf = RandomForestClassifier(n_estimators=100) # initialize
rf.fit(trainArr, trainRes) # fit the data to the algorithm


RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [749]:
testArr = result_test.as_matrix(cols)
results = rf.predict(testArr)
relevance = results
relevance.shape

(166693,)

In [770]:
index = list(range(166694))[1:]

In [771]:
df = pd.DataFrame(index = index, data=relevance)
df

,0
1,2.33
2,2.33
3,3.00
4,3.00
5,3.00
6,2.33
7,3.00
8,3.00
9,3.00
10,3.00


In [772]:
df.to_csv("/Users/shijiabian/Dropbox/Duke Statistics Courses/Spring 2016 Duke Course/STA 571 Adv ML/Final Project/submission/sub_4_20.csv")

In [114]:
oneT = str(result["product_title"][2]).lower()
m=re.findall(r"premium", str(result["product_title"][2]).lower())
len(m)

1

In [125]:
for w in (str(result["search_term"][1]).split()):
        print w

l
bracket


In [130]:
# split the words in the searching query
[w.lower() for w in (str(result["search_term"]).split())]

['0',
 'angle',
 'bracket',
 '1',
 'l',
 'bracket',
 '2',
 'deck',
 'over',
 '3',
 'rain',
 'shower',
 'head',
 '4',
 'shower',
 'only',
 'faucet',
 '5',
 'convection',
 'otr',
 '6',
 'microwave',
 'over',
 'stove',
 '7',
 'microwaves',
 '8',
 'emergency',
 'light',
 '9',
 'mdf',
 '3/4',
 'name:',
 'search_term,',
 'dtype:',
 'object']

In [133]:
def split_search(value) :
    res = [w.lower() for w in (str(value).split())]
    return res

In [217]:
ss = result["search_term"].apply(split_search) 
result['search_split'] = pd.Series(ss, index=result.index)

NameError: name 'search_split' is not defined

In [254]:
for index, row in result.iterrows():
    each_word_len = [len(w) for w in row["search_split"]]
    each_word_len = [mm for mm in each_word_len if mm > 1]
    print (each_word_len)

[5, 7]
[7]
[4, 4]
[4, 6, 4]
[6, 4, 6]
[10, 3]
[9, 4, 5]
[10]
[9, 5]
[3, 3]


In [157]:
' '.join(ss[1])

'l bracket'

In [146]:
oneT = str(result["product_title"][2]).lower()
m=re.findall(ss[1][1], "this is a bracket")
len(m)

1

In [176]:
re.findall(ss[0][0], result["product_description"][0])

[u'angle', u'angle', u'angle']

In [181]:
split_search_term = result["search_term"].apply(split_search) 
text = "this is a bracket"
for 
m=re.findall(split_search_term[1], "this is a bracket")

TypeError: unhashable type: 'list'

In [209]:
ll = [len(re.findall(w, "this is a bracket")) for w in split_search_term[1]]
ll
result.shape[1]

8

In [216]:


nrow = result.shape[0]
match_word = [0]*nrow

for index, row in result.iterrows():
    ll = [len(re.findall(w, row["product_description"])) for w in row["search_split"]]
    match_word[index] = sum(ll)
    
print (match_word)

[3, 20, 4, 0, 1, 2, 8, 0, 2, 1]
